In [21]:
!pip install langchain
!pip install bitsandbytes
!pip install accelerate
!pip install transformers
!pip install sentence_transformers

In [22]:
!pip install unstructured

In [23]:
!pip install -U langchain-community

In [1]:
from langchain.document_loaders import UnstructuredURLLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from transformers import AutoTokenizer, AutoModelForCausalLM
import transformers
from langchain import HuggingFacePipeline
from huggingface_hub import notebook_login
import torch

# Extract data from url

In [2]:
import nltk
urls = [
    "https://ineuron.ai/"
]

In [9]:
print(nltk.__version__)

3.8.1


In [11]:
# import nltk
# nltk.download()

In [3]:
loader=UnstructuredURLLoader(urls=urls)
data = loader.load()

In [4]:
data

[Document(metadata={'source': 'https://ineuron.ai/'}, page_content='Learning with iNeuron made\n\nTake your career to the next level with industry ready programs, An entire learning ecosystem at your fingertips to make learning fun. Choose from a range of tech programs and make your next big career switch.\n\n55%\n\nAverage Salary Hike\n\n400+\n\nDifferent Courses\n\n10000+\n\nCareer Transitions\n\n400+\n\nHiring Partners\n\nLIVE NOW\n\nSupport System\n\nOur support system is live again, this time it is bigger, better and faster. Experience a tech community like never seen before\n\nOur Courses\n\nSuccess Stories\n\nFresher\n\nAbhisekh Bhuyan\n\nMLOps engineer\n\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\n\nFrom\n\nFresher\n\nTo\n\n79% Increment\n\nSubham Kanungo\n\nAssociate Data Scientist\n\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\n\nFrom\n\nTo\n\n1

# Chunkings 

In [5]:
text_splitter = CharacterTextSplitter(separator='\n', chunk_size = 1000, chunk_overlap=200)

In [6]:
text_chunk = text_splitter.split_documents(data)

In [7]:
len(text_chunk)

6

In [8]:
text_chunk[0]

Document(metadata={'source': 'https://ineuron.ai/'}, page_content='Learning with iNeuron made\nTake your career to the next level with industry ready programs, An entire learning ecosystem at your fingertips to make learning fun. Choose from a range of tech programs and make your next big career switch.\n55%\nAverage Salary Hike\n400+\nDifferent Courses\n10000+\nCareer Transitions\n400+\nHiring Partners\nLIVE NOW\nSupport System\nOur support system is live again, this time it is bigger, better and faster. Experience a tech community like never seen before\nOur Courses\nSuccess Stories\nFresher\nAbhisekh Bhuyan\nMLOps engineer\nI got job as an MLOps engineer at synapsica at 13 LPA PPO because of "End to End projects MLOps" from iNeuron.\nFrom\nFresher\nTo\n79% Increment\nSubham Kanungo\nAssociate Data Scientist\nI just joined EY as data analyst. It would not be possible with the support of Krish sir and Sudhanshu sir.\nFrom\nTo\n100% Increment\nSayan Saha\nSoftware Engineer - Python')

# Embedding model 

In [17]:
embeddings=HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')

ImportError: Could not import sentence_transformers python package. Please install it with `pip install sentence-transformers`.

In [26]:
!pip install tf-keras

  Obtaining dependency information for tf-keras from https://files.pythonhosted.org/packages/21/8b/75f7572ec0273ed8da50bc19defe08aaaafcc15fda3407db53f49acec814/tf_keras-2.17.0-py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow<2.18,>=2.17 from https://files.pythonhosted.org/packages/0f/a5/89a9bdae9f241cad167edb6c890ef2b4c842c5ae81058a2fbfd702dba9bf/tensorflow-2.17.0-cp311-cp311-macosx_12_0_arm64.whl.metadata
  Obtaining dependency information for tensorboard<2.18,>=2.17 from https://files.pythonhosted.org/packages/0a/32/2e8545fb0592f33e3aca5951e8b01008b76d61b440658cbdc37b4eaebf0b/tensorboard-2.17.0-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 13.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.2/236.2 MB 9.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 18.3 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard

In [10]:
!pip install sentence-transformers